In [1]:
import numpy as np
import pandas as pd

In [2]:
def read_data_set(file_path,encode='cp850'):
    df = pd.read_csv(file_path,encoding=encode)
    return df

### Read Data Set

In [3]:
df_data_set_1=read_data_set("D:/Fourth_Year/Second_Term/BD/KickStarter-Success-Prediction/ks-projects-201612.csv")
df_data_set_2=read_data_set("D:/Fourth_Year/Second_Term/BD/KickStarter-Success-Prediction/ks-projects-201801.csv")

C:\Users\rorod\AppData\Local\Temp\ipykernel_31572\474565683.py:2: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,encoding=encode)


### Remove Repeated Columns and rename them

In [4]:
def remove_unnamed_columns(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

In [5]:
def remove_repeated_columns(df,column):
    df=df.drop([column], axis=1)
    return df

In [6]:
def rename_columns_df(df):
    columns=list(df_data_set_1.columns)
    for column in columns:
        df.rename(columns = { column:column[:-1]}, inplace = True)
    return df


In [7]:
df_data_set_1=remove_unnamed_columns(df_data_set_1)

In [8]:
df_data_set_2=remove_repeated_columns(df_data_set_2,"usd_pledged_real")
df_data_set_2=remove_repeated_columns(df_data_set_2,"usd_goal_real")

In [9]:
df_data_set_1=rename_columns_df(df_data_set_1)

### Merge Data Set

In [10]:
frames = [df_data_set_1, df_data_set_2]
df = pd.concat(frames)

### Preprocessing Data Set

In [11]:
#get null percentage
df.isnull().sum()/df.shape[0]

ID               0.000000
name             0.000011
category         0.000007
main_category    0.000000
currency         0.000000
deadline         0.000000
goal             0.000000
launched         0.000000
pledged          0.000000
state            0.000000
backers          0.000000
country          0.000000
usd pledged      0.010801
dtype: float64

In [12]:
# drop all nulls remaining in name & category
df = df.dropna(axis=0, subset=['name', 'category'])

In [13]:
# convert usd pledged to float values
df.loc[:,'usd pledged'] = pd.to_numeric(df['usd pledged'], downcast='float', errors='coerce')
# convert goal to float
df['goal'] = pd.to_numeric(df.goal, downcast='float', errors='coerce')

C:\Users\rorod\AppData\Local\Temp\ipykernel_31572\2040241626.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'usd pledged'] = pd.to_numeric(df['usd pledged'], downcast='float', errors='coerce')


In [14]:
# fill all nan with 0
df = df.fillna(value=0.0)

In [15]:
# check nan in country
df[~df.country.str.contains('^[A-Z]{2}$', case=False)].country.value_counts()

N,0"    3797
N,"0    3790
0        102
1         72
3         45
        ... 
213        1
126        1
582        1
26         1
43         1
Name: country, Length: 142, dtype: int64

In [16]:
# Replace null countries with None
replace = df[~df.country.str.contains('^[A-Z]{2}$', case=False)].country.unique().tolist()
df.loc[:,'country'] = df.country.replace(to_replace=replace, value='None')

In [17]:
df.loc[:,'backers'] = pd.to_numeric(df.backers, errors='coerce', downcast='integer')

C:\Users\rorod\AppData\Local\Temp\ipykernel_31572\1622262248.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'backers'] = pd.to_numeric(df.backers, errors='coerce', downcast='integer')


In [18]:
df = df.dropna(axis=0, subset=['backers'])

In [19]:
df.head(10)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000.0,2015-08-11 12:12:28,0,failed,0.0,GB,0.00
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000.0,2013-01-12 00:20:50,220,failed,3.0,US,220.00
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000.0,2012-03-17 03:24:11,1,failed,1.0,US,1.00
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500.0,2015-07-04 08:35:03,1283,canceled,14.0,US,1283.00
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000.0,2016-02-26 13:38:27,52375,successful,224.0,US,52375.00
5,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21 18:30:44,1000.0,2014-12-01 18:30:44,1205,successful,16.0,US,1205.00
6,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17 19:05:12,25000.0,2016-02-01 20:05:12,453,failed,40.0,US,453.00
7,1000034518,SPIN - Premium Retractable In-Ear Headphones w...,Product Design,Design,USD,2014-05-29 18:14:43,125000.0,2014-04-24 18:14:43,8233,canceled,58.0,US,8233.00
8,100004195,STUDIO IN THE SKY - A Documentary Feature Film...,Documentary,Film & Video,USD,2014-08-10 21:55:48,65000.0,2014-07-11 21:55:48,6240.57,canceled,43.0,US,6240.57
9,100004721,Of Jesus and Madmen,Nonfiction,Publishing,CAD,2013-10-09 18:19:37,2500.0,2013-09-09 18:19:37,0,failed,0.0,CA,0.00


In [22]:
df["state"].value_counts()

failed                 365934
successful             247037
canceled                71133
live                     7227
undefined                7117
suspended                3323
2011-06-04 01:17:48         1
2016-06-28 21:36:59         1
2016-02-18 21:40:23         1
2016-02-22 21:57:30         1
2015-09-08 22:08:39         1
2013-01-03 03:45:14         1
2015-10-10 01:00:00         1
2015-01-28 13:53:56         1
2016-02-15 17:24:27         1
Name: state, dtype: int64

In [25]:
failed=df.loc[df['state'] == "failed"  ]

In [26]:
successed =df.loc[df['state'] == "successful"]

In [27]:
new_frames = [successed,failed]
new_df = pd.concat(new_frames)

In [28]:
new_df["state"].value_counts()

failed        365934
successful    247037
Name: state, dtype: int64